In [1]:
import pandas as pd
import random
import numpy as np
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QGridLayout, QFileDialog, QLabel, QComboBox
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5 import uic, QtGui

In [2]:
import hashlib
class anonymous() :
    def __init__(self,df):
        self.df = df
        self.backup = backup
        
    def SHA256withSalt(self, column, salt):
        """
            SHA256withSalt : sha256 해시 함수 + salt
        """
        self.backup = self[column]
        SHA256 = hashlib.sha256() # sha256 hash 객체 생성
        col = []
        for i in range(len(self.index)):
            SHA256.update((str(self.loc[i][column])+str(salt)).encode('euc-kr'))
            col.append(SHA256.hexdigest()) # sha 256 해시에 대한 16진수 값 산출
        self[column] = col
        after = self
        return after
    
    def delete_col(self, column):
        """
            delete_col : 열 삭제
            column : 삭제할 column명
        """
        self.backup = self[column]
        after = self.drop(column, axis=1)
        return after
    
    def delete_row(self, rows):
        """
            delete_row : 행 삭제
            - rows : 삭제할 row명
        """
        self.backup = self.iloc[rows]
        del_df = self.drop(rows, axis=0)
        return del_df
    
    def delete_partial(self, column, num):
        """
            delete_partial : 부분 삭제
            - column : 부분 삭제할 column명
            - num : num번째 이후 값 삭제
        """
        self.backup = self[column]
        splitData = self[column].str.split(' ')
        
        col = []
        for i in splitData:
            i = i[:num]
            col.append(" ".join(i))
        self[column] = col
        after = self
        return after

In [3]:
form_class = uic.loadUiType("third_som_ver.ui")[0]

#화면을 띄우는데 사용되는 Class 선언
class WindowClass(QMainWindow, form_class) :    
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        
        # 로고 이미지
        qPixmapVar = QPixmap()
        qPixmapVar.load("logo2.jpg")
        self.Logo.setPixmap(qPixmapVar)
        
        self.open_button.clicked.connect(self.open_file) # open_button 클릭 관련
        self.apply_button.clicked.connect(self.apply) # '비식별 처리' apply 버튼 클릭 곤련
        self.save_button.clicked.connect(self.save_file) # save_button 클릭 관련
        
    def open_file(self) : # 파일 불러오기
        open_name = QFileDialog.getOpenFileName(self, 'Open File', './')        
        data = pd.read_csv(open_name[0], encoding='utf-8')
        
        global after # 비식별 처리 후 데이터를 저장할 전역변수 after
        after = data
        
        # table_1 widget 행, 열 추가
        self.table_1.setRowCount(len(data))
        self.table_1.setColumnCount(len(list(data)))
        self.table_1.setHorizontalHeaderLabels(list(data))

        # table_1 widget에 불러온 파일의 데이터 추가
        for i in range (len(data)) :
            for j in range (len(list(data))) :
                self.table_1.setItem(i, j, QTableWidgetItem(format(data.values.tolist()[i][j])))
    
    def apply(self) : # 임시 총계처리 평균
        row = self.table_1.currentRow()
        column = self.table_1.currentColumn()
        func = self.comboBox.currentText()
        
        if func == 'delete_col':
            for i in range (len(after)) :
                self.table_1.takeItem(i,column) # 비식별한 열 출력
                
        elif func == 'delete_partial':
            tmp = anonymous.delete_partial(after, after.columns[column], 1)
            
            for i in range(len(tmp)) :
                self.table_1.setItem(i, column, QTableWidgetItem(format(tmp.values.tolist()[i][column]))) # 비식별한 열 출력
                
        elif func == 'hash':
            tmp = anonymous.SHA256withSalt(after,after.columns[column],"something")
            
            for i in range(len(tmp)) :
                self.table_1.setItem(i, column, QTableWidgetItem(format(tmp.values.tolist()[i][column]))) # 비식별한 열 출력

        
    def save_file(self) : # 비식별처리 후 파일 저장
        save_name = QFileDialog.getSaveFileName(self, 'Open File', './')
        after.to_csv(save_name[0], encoding='utf-8', index=False)
        
if __name__ == "__main__" :
    app = QApplication(sys.argv) 
    myWindow = WindowClass() 
    myWindow.show()
    app.exec_()

C:\Users\somin\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # This is added back by InteractiveShellApp.init_path()
